### CÁC PHƯƠNG PHÁP AUGEMENTATION

1. JITTERING (thêm nhiễu vào các keypoints)
- Thêm nhiễu guassian (norm distribution) vào các keypoints
- giúp mô hình không bị phụ thuộc vào vị trí chính xác của keypoints
- giúp mô hình học được các biến thể của keypoints
- công thức: skeleton(aug) = skeleton + N(0, sigma^2) với N(0, sigma^2) là phân phối Guassian có trung bình 0 và độ lệch chuẩn sigma
- Dùng khi nào? Khi muốn mô hình học được sự biến đổi nhỏ trong tọa độ keypoints, khi dữ liệu quá sạch và dễ bị overfitting
- VD: ban đầu: (0.5, 0.3, 0.2) --> jittering (với noise nhỏ): (0.502, 0.295, 0.205)
2. Scaling (phóng to/ thu nhỏ toàn bộ skeleton)
- Phóng to hoặc thu nhỏ keypoints theo một hệ số ngẫu nhiên.
- Giúp mô hình không bị phụ thuộc vào kích thước người tập yoga.
- Công thức: skeleton(aug) = skeleton * scale_factor (trong này set là trong khoảng 0.9 và 1.1)
- Dùng khi nào? Khi muốn mô hình học được sự biến đổi về kích thước của người tập yoga (cao thấp, mập ốm,...) --> không bị ảnh hưởng bởi kích thước tuyệt đối
- VD: ban đầu: (0.5, 0.3, 0.2) --> scaling (scale_factor = 1.1): (0.55, 0.33, 0.22)
3. Rotation (Xoay toàn bộ skeleton quanh trung tâm)
- Xoay khung xương quanh trung tâm theo góc ngẫu nhiên (-15 đến 15 độ)
- Giúp mô hình hiểu được các tư thế nhìn từ nhiều góc độ khác nhau.
- Công thức: 
    * R = [[cos(theta), -sin(theta)], [sin(theta), cos(theta)]] với theta là góc xoay
    * skeleton(aug) = R * (skeleton - center)*R + center
- Dùng khi nào? Khi muốn mô hình học được sự biến đổi về góc nhìn của người tập yoga, phòng khi data chỉ thu được từ một góc độ cố định
- VD: Nếu điểm tay ban đầu: (0.5, 0.3) --> xoay 10 độ (0.48, 0.32)
4. Horizontal Flip (Lật keypoints trái/phải)
- Lật toàn bộ tư thế theo trục X
- Giúp mô hình hiểu cả tư thế từ cả hai phía trái phải của cùng một động tác
- công thức: skeleton(aug) = [:, :, 0] = 1 - skeleton[:, :, 0]
- Dùng khi nào? Khi muốn mô hình học được sự biến đổi về trái phải của người tập yoga, khi dữ liệu chỉ có một phiên bản của động tác
- VD: ban đầu (0.2, 0.5) --> flip (0.8, 0.5)
5. Temporal Warping (Tăng/Giảm tốc độ chuyển động)
- Tăng hoặc giảm tốc độ của video mà không làm mất động tác.
- Giúp mô hình hiểu được động tác thực hiện nhanh/chậm khác nhau.
- công thức: 
    * num_frames = T * warp_factor
    * skeleton(aug) = skeleton[indices]
    * với warp_factor (0.8, 1.2) là hệ số tốc độ, indices là các chỉ số của frame sau khi tăng/giảm tốc độ
- Dùng khi nào? Khi động tác có thể thực hiện với tốc độ khác nhau, khi video có frame_rate không đồng đều
- VD: ban đầu: 10 frames --> giảm tốc độ (warp_factor = 1.2): 12 frames
6. Time Masking (Ẩn một số frame)
- Loại bỏ một số frame ngẫu nhiên, giả lập video bị mất frame hoặc hành động không liên tục.
- Giúp mô hình học cách đối phó với dữ liệu không liên tục.
- Dùng khi nào? Khi video có tốc độ ghi hình không ổn định, khi muốn mô hình không bị phụ thuộc vào toàn bộ chuỗi frame.
- VD: ban đầu: [F1, F2, F3, F4, F5] --> Sau masking: [F1, 0, F3, 0, F5]
7. Frame Interpolation (Nội suy khung hình)
- Tạo thêm frame nội suy giữa các frame thực tế, giúp mô hình học mượt hơn.
- Giả lập dữ liệu có tốc độ quay cao hơn.
- Dùng khi nào? Khi video bị ít frame, cần tạo thêm frame để training, khi muốn mô hình học được chuyển động mượt mà hơn.
- VD: ban đầu: [F1, F2, F3] --> Interpolation: [F1, F1.5, F2, F2.5, F3]

==================================KẾT LUẬN================================
- Nếu muốn tăng độ đa dạng, thử Jittering + Rotation + Scaling.
- Nếu dữ liệu bị thiếu keypoints, thử Keypoint Masking + Time Masking.
- Nếu dữ liệu có tốc độ khác nhau, thử Temporal Warping + Frame Interpolation.

### AUGMENT THẾ NÀO CHO ĐÚNG?
1. Augmentation phải giữ nguyên bản chất dữ liệu
- Augmentation không được làm mất đi cấu trúc động tác.
    * Horizontal Flip có thể sai nếu động tác chỉ có một chiều nhất định. (Sai)
    * Rotation quá mạnh (>30°) có thể làm sai tư thế. (Sai)
    * Jittering chỉ nên thêm nhiễu nhỏ (±0.01) để tránh làm thay đổi vị trí động tác. (Đúng)
    * Rotation chỉ nên xoay trong khoảng ±15° để giữ đúng cấu trúc khung xương. (Đúng)
2. Augmentation phải đa dạng nhưng không gây nhiễu quá mức
- Không nên áp dụng quá nhiều augmentation cùng lúc, vì sẽ làm dữ liệu mất đi sự đồng nhất.
- Cần thử nghiệm và chọn những augmentation thực sự hữu ích.
- Nếu dataset nhỏ → Nên thử nhiều loại augmentation.
- Nếu dataset đã lớn → Chỉ dùng augmentation nhẹ như Jittering, Rotation, Scaling.
    * Jittering (0.01) + Rotation (±15°) + Temporal Warping (0.9 - 1.1). (Đúng)
    * Jittering (0.1) + Rotation (±45°) + Scaling (0.5 - 1.5) → Quá mạnh, có thể làm hỏng dữ liệu. (Sai)
3. Augmentation phải phù hợp với bài toán
- Cần hiểu rõ bài toán và dữ liệu để chọn augmentation phù hợp.
- Đối với Yoga: Tư thế có thể rất quan trọng, Horizontal Flip có thể không hợp lý.
- Đối với hành động thể thao: Temporal Warping có thể giúp mô hình học tốc độ thực hiện khác nhau. 
- Đúng:
    * Nếu tư thế Yoga có thể lật trái/phải → Dùng Horizontal Flip.
    * Nếu cần học tốc độ thực hiện khác nhau → Dùng Temporal Warping.
    * Nếu muốn mô hình mạnh hơn với dữ liệu nhiễu → Dùng Keypoint Masking.

In [ ]:
import cv2
import numpy as np
import os
import mediapipe as mp

mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

def jittering(skeleton, noise_level=0.01):
    """ Thêm nhiễu ngẫu nhiên vào keypoints """
    noise = np.random.normal(loc=0, scale=noise_level, size=skeleton.shape)
    return skeleton + noise

def scaling(skeleton, scale_range=(0.9, 1.1)):
    """ Phóng to/thu nhỏ keypoints """
    scale_factor = np.random.uniform(scale_range[0], scale_range[1])
    return skeleton * scale_factor

def rotation(skeleton, angle_range=(-15, 15)):
    """ Xoay keypoints quanh trung tâm """
    angle = np.radians(np.random.uniform(angle_range[0], angle_range[1]))
    cos_val, sin_val = np.cos(angle), np.sin(angle)

    # Tạo ma trận xoay
    rotation_matrix = np.array([[cos_val, -sin_val], [sin_val, cos_val]])

    # Chỉ xoay x, y (không xoay z)
    skeleton[:, :, :2] = np.dot(skeleton[:, :, :2] - np.mean(skeleton[:, :, :2], axis=0), rotation_matrix) + np.mean(skeleton[:, :, :2], axis=0)
    
    return skeleton

def horizontal_flip(skeleton):
    """ Lật keypoints trái/phải """
    skeleton[:, :, 0] = 1 - skeleton[:, :, 0]  # Đảo ngược trục x
    return skeleton

def temporal_warping(skeleton, warp_factor_range=(0.8, 1.2)):
    """ Tăng/giảm tốc độ chuyển động (thay đổi số frame) """
    warp_factor = np.random.uniform(warp_factor_range[0], warp_factor_range[1])
    num_frames = int(skeleton.shape[0] * warp_factor)
    indices = np.linspace(0, skeleton.shape[0] - 1, num_frames, dtype=int)
    return skeleton[indices]

def time_masking(skeleton, mask_ratio=0.2):
    """ Ngẫu nhiên bỏ qua một số frame trong video """
    T = skeleton.shape[0]
    num_mask = int(T * mask_ratio)
    mask_indices = np.random.choice(T, num_mask, replace=False)
    skeleton[mask_indices] = 0  # Gán toàn bộ frame đó về 0
    return skeleton

def frame_interpolation(skeleton):
    """ Thêm frame nội suy giữa các frame hiện tại """
    T, V, C = skeleton.shape
    new_T = T * 2 - 1  # Tăng gấp đôi số frame
    interpolated_skeleton = np.zeros((new_T, V, C))

    for i in range(T - 1):
        interpolated_skeleton[2 * i] = skeleton[i]
        interpolated_skeleton[2 * i + 1] = (skeleton[i] + skeleton[i + 1]) / 2  # Nội suy
    
    interpolated_skeleton[-1] = skeleton[-1]
    return interpolated_skeleton

In [ ]:
def extract_skeleton_with_augmentation(input_folder, output_folder, fps=10, augment=True):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for pose_name in os.listdir(input_folder):
        pose_path = os.path.join(input_folder, pose_name)
        output_pose_path = os.path.join(output_folder, pose_name)

        if not os.path.exists(output_pose_path):
            os.makedirs(output_pose_path)

        for file in os.listdir(pose_path):
            if file.endswith(".mp4"):
                video_path = os.path.join(pose_path, file)
                output_file = os.path.join(output_pose_path, file.replace(".mp4", ".npy"))

                if os.path.exists(output_file):
                    continue  # Bỏ qua nếu đã trích xuất

                cap = cv2.VideoCapture(video_path)
                video_fps = cap.get(cv2.CAP_PROP_FPS)
                total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                selected_frames = np.arange(0, total_frames, step=int(video_fps / fps), dtype=int)
                skeleton_data = []

                for idx in selected_frames:
                    cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
                    ret, frame = cap.read()
                    if not ret:
                        continue

                    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    results = pose.process(frame_rgb)

                    keypoints = []
                    if results.pose_landmarks:
                        for lm in results.pose_landmarks.landmark:
                            keypoints.append([lm.x, lm.y, lm.z])
                    else:
                        keypoints = [[0, 0, 0]] * 33  # Nếu không nhận diện được, gán 0

                    skeleton_data.append(keypoints)

                cap.release()
                skeleton_data = np.array(skeleton_data)  # Shape: (num_extracted_frames, 33, 3)

                # Thực hiện augmentation nếu bật flag `augment`
                if augment:
                    skeleton_aug1 = jittering(skeleton_data)
                    skeleton_aug2 = scaling(skeleton_data)
                    skeleton_aug3 = rotation(skeleton_data)
                    skeleton_aug4 = horizontal_flip(skeleton_data)
                    skeleton_aug5 = temporal_warping(skeleton_data)
                    skeleton_aug6 = time_masking(skeleton_data)
                    skeleton_aug7 = frame_interpolation(skeleton_data)

                    # Lưu augmentation vào file mới
                    np.save(output_file.replace(".npy", "_jitter.npy"), skeleton_aug1)
                    np.save(output_file.replace(".npy", "_scale.npy"), skeleton_aug2)
                    np.save(output_file.replace(".npy", "_rotate.npy"), skeleton_aug3)
                    np.save(output_file.replace(".npy", "_flip.npy"), skeleton_aug4)
                    np.save(output_file.replace(".npy", "_warp.npy"), skeleton_aug5)
                    np.save(output_file.replace(".npy", "_mask.npy"), skeleton_aug6)
                    np.save(output_file.replace(".npy", "_interpolate.npy"), skeleton_aug7)

                # Lưu file gốc
                np.save(output_file, skeleton_data)

                print(f"Processed {video_path} -> {output_file} | Extracted Frames: {skeleton_data.shape[0]}")

In [ ]:
# Chạy trích xuất skeleton không có augmentation
extract_skeleton_with_augmentation("Yoga_9gb", "Skeleton_data_9gb", fps=10, augment=False)

In [ ]:
# Chạy trích xuất skeleton với augmentation
extract_skeleton_with_augmentation("Yoga_9gb", "Skeleton_data_9gb_augmented", fps=10, augment=True)

PHƯƠNG PHÁP MEAN CENTERING NORMALIZATION
- Tính trung bình của tất cả keypoints trong toàn bộ video trên tọa độ x và y.
- Dịch tất cả keypoints về trung tâm (0,0) bằng cách trừ đi trung bình này.

Mục đích: 
- Giúp mô hình không bị ảnh hưởng bởi vị trí tuyệt đối của đối tượng.
    * Nếu người thực hiện động tác đứng lệch trái/phải trong video, tọa độ tuyệt đối sẽ khác, nhưng động tác vẫn giống nhau.
    * Normalize giúp mô hình chỉ tập trung vào hình dạng động tác, không phải vị trí. 
- Tạo sự đồng nhất giữa các video
    * Nếu video 1 có người ở góc trên bên phải, video 2 có người ở trung tâm, thì mô hình có thể gặp khó khăn khi học.
    * Normalize giúp đồng nhất dữ liệu, làm cho mô hình dễ học hơn. 
- Loại bỏ ảnh hưởng của vị trí camera
    * Nếu camera đặt ở nhiều góc khác nhau, tọa độ keypoints có thể lệch nhưng động tác vẫn giống nhau.
    * Chuẩn hóa giúp mô hình không phụ thuộc vào góc nhìn camera.     

In [ ]:
import os
import numpy as np

"""
input shape: (num_frames, num_keypoints, 3) --> (T, 33, 3)
    - T: số lượng frame
    - 33: số lượng keypoints
    - 3: tọa độ x, y, z của keypoints
Công thức toán học: 
    - mean(x) = 1/T*33 * sum(t=1->T, sum(i=1->33, x_t,i))
    - mean(y) = 1/T*33 * sum(t=1->T, sum(i=1->33, y_t,i))
    - skeletion(t,i,x) = x_t,i - mean(x)
    - skeletion(t,i,y) = y_t,i - mean(y)
    --> Mỗi keypoint (x, y) được trừ đi trung bình của toàn bộ keypoints
"""

def normalize_skeleton(skeleton):
    """Chuẩn hóa skeleton về trung tâm bằng cách trừ đi tọa độ trung bình."""
    mean_pose = np.mean(skeleton[:, :, :2], axis=(0, 1))  # Trung bình trên trục x, y
    skeleton[:, :, :2] -= mean_pose  # Dịch keypoints về trung tâm
    return skeleton

input_folder = "Skeleton_data_9gb"
# input_folder = "Skeleton_data_9gb_augmented"

if not os.path.exists(input_folder):
    raise FileNotFoundError(f"Thư mục {input_folder} không tồn tại!")

for pose_name in os.listdir(input_folder):
    pose_path = os.path.join(input_folder, pose_name)
    
    if not os.path.isdir(pose_path):  # Bỏ qua nếu không phải thư mục
        continue

    for file in os.listdir(pose_path):
        if file.endswith(".npy"):
            file_path = os.path.join(pose_path, file)
            
            # Load skeleton
            skeleton = np.load(file_path)

            # Chuẩn hóa
            skeleton = normalize_skeleton(skeleton)

            # Ghi đè file sau khi chuẩn hóa
            np.save(file_path, skeleton)
            print(f"✅ Normalized: {file_path}")

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class YogaSkeletonDataset(Dataset):
    def __init__(self, root_dir, classes, T_max=100):
        self.root_dir = root_dir
        self.classes = classes
        self.video_paths = []
        self.labels = []
        self.T_max = T_max

        for label, cls in enumerate(classes):
            class_dir = os.path.join(root_dir, cls)
            for file in os.listdir(class_dir):
                if file.endswith(".npy"):
                    self.video_paths.append(os.path.join(class_dir, file))
                    self.labels.append(label)

    def pad_skeleton(self, skeleton):
        T, V, C = skeleton.shape
        if T < self.T_max:
            pad = np.zeros((self.T_max - T, V, C))
            skeleton = np.concatenate((skeleton, pad), axis=0)
        else:
            skeleton = skeleton[:self.T_max]
        return skeleton

    def __getitem__(self, idx):
        skeleton = np.load(self.video_paths[idx])
        skeleton = self.pad_skeleton(skeleton)
        label = self.labels[idx]
        return torch.tensor(skeleton, dtype=torch.float32), torch.tensor(label, dtype=torch.long)

    def __len__(self):
        return len(self.video_paths)

# Tạo DataLoader
classes = ["Garland_Pose", "Happy_Baby_Pose", "Head_To_Knee_Pose", "Lunge_Pose",
           "Mountain_Pose", "Plank_Pose", "Raised_Arms_Pose", "Seated_Forward_Bend",
           "Staff_Pose", "Standing_Forward_Bend"]

train_dataset = YogaSkeletonDataset("Skeleton_data_9gb", classes, T_max=100)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

# train_dataset = YogaSkeletonDataset("Skeleton_data_9gb_augmented", classes, T_max=100)
# train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

# Kiểm tra dữ liệu
skeleton_sample, label_sample = train_dataset[0]
print("Sample shape:", skeleton_sample.shape)  # Expect (100, 33, 3)
print("Label:", classes[label_sample.item()])

In [ ]:
dataset_path = "Skeleton_data_9gb"  # Chọn bộ gốc
# dataset_path = "Skeleton_data_9gb_augmented"  # Chọn bộ augmented

video_paths = []
labels = []
classes = sorted(os.listdir(dataset_path))  # Danh sách động tác

train_videos = []
train_labels = []
valid_videos = []
valid_labels = []

for label, cls in enumerate(classes):
    class_dir = os.path.join(dataset_path, cls)
    if not os.path.isdir(class_dir):
        continue

    # Lấy danh sách file .npy trong thư mục của động tác hiện tại
    files = [os.path.join(class_dir, f) for f in os.listdir(class_dir) if f.endswith(".npy")]

    # Xáo trộn danh sách file để đảm bảo train/valid ngẫu nhiên
    np.random.shuffle(files)

    # Chia theo tỷ lệ 80% train / 20% validation
    split_idx = int(0.8 * len(files))
    train_videos.extend(files[:split_idx])
    train_labels.extend([label] * split_idx)
    valid_videos.extend(files[split_idx:])
    valid_labels.extend([label] * (len(files) - split_idx))

# Tạo dataset và dataloader cho train và validation
train_dataset = YogaSkeletonDataset(train_videos, train_labels, T_max=100)
valid_dataset = YogaSkeletonDataset(valid_videos, valid_labels, T_max=100)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=4, shuffle=False)

# Kiểm tra số lượng mẫu trong train/valid
print(f"📌 Tổng số mẫu: {len(train_dataset) + len(valid_dataset)}")
print(f"✅ Train set: {len(train_dataset)} samples")
print(f"✅ Validation set: {len(valid_dataset)} samples")

# Kiểm tra một mẫu trong train set
skeleton_sample, label_sample = train_dataset[0]
print("Sample shape:", skeleton_sample.shape)  # Kỳ vọng: (100, 33, 3)
print("Label:", classes[label_sample.item()])

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.nn as gnn
from torch_geometric.data import Data

class YogaGCN(nn.Module):
    def __init__(self, in_channels=3, hidden_dim=64, num_classes=10):
        super(YogaGCN, self).__init__()
        self.conv1 = gnn.GCNConv(in_channels, hidden_dim)
        self.conv2 = gnn.GCNConv(hidden_dim, hidden_dim)
        self.conv3 = gnn.GCNConv(hidden_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index).relu()
        x = self.conv3(x, edge_index).relu()
        x = torch.mean(x, dim=0)  # Pooling để lấy đặc trưng toàn video
        x = self.fc(x)
        return x

# V2: thử tăng kích thước nếu model v1 lỏ    
# class YogaGCN(nn.Module):
#     def __init__(self, in_channels=3, hidden_dim=128, num_classes=10):
#         super(YogaGCN, self).__init__()
#         self.conv1 = gnn.GCNConv(in_channels, hidden_dim)
#         self.conv2 = gnn.GCNConv(hidden_dim, hidden_dim)
#         self.conv3 = gnn.GCNConv(hidden_dim, hidden_dim)
#         self.conv4 = gnn.GCNConv(hidden_dim, hidden_dim)  # Thêm một lớp nữa
#         self.fc = nn.Linear(hidden_dim, num_classes)

#     def forward(self, x, edge_index):
#         x = self.conv1(x, edge_index).relu()
#         x = self.conv2(x, edge_index).relu()
#         x = self.conv3(x, edge_index).relu()
#         x = self.conv4(x, edge_index).relu()
#         x = torch.mean(x, dim=0)
#         x = self.fc(x)
#         return x

# V3: thêm dropout
# class YogaGCN(nn.Module):
#     def __init__(self, in_channels=3, hidden_dim=64, num_classes=10):
#         super(YogaGCN, self).__init__()
#         self.conv1 = gnn.GCNConv(in_channels, hidden_dim)
#         self.conv2 = gnn.GCNConv(hidden_dim, hidden_dim)
#         self.conv3 = gnn.GCNConv(hidden_dim, hidden_dim)
#         self.fc = nn.Linear(hidden_dim, num_classes)

#     self.dropout = nn.Dropout(0.3)  # 30% dropout

#     def forward(self, x, edge_index):
#         x = self.conv1(x, edge_index).relu()
#         x = self.dropout(x)
#         x = self.conv2(x, edge_index).relu()
#         x = self.dropout(x)
#         x = self.conv3(x, edge_index).relu()
#         x = torch.mean(x, dim=0)
#         x = self.fc(x)
#         return x
    
    
# V4: Dùng GAT (Graph Attention Network), chậm hơn cần GPU mạnh hơn
# class YogaGAT(nn.Module):
#     def __init__(self, in_channels=3, hidden_dim=128, num_classes=10):
#         super(YogaGAT, self).__init__()
#         self.conv1 = gnn.GATConv(in_channels, hidden_dim, heads=4, concat=True)
#         self.conv2 = gnn.GATConv(hidden_dim * 4, hidden_dim, heads=4, concat=True)
#         self.conv3 = gnn.GATConv(hidden_dim * 4, hidden_dim, heads=4, concat=True)
#         self.fc = nn.Linear(hidden_dim * 4, num_classes)

#     def forward(self, x, edge_index):
#         x = self.conv1(x, edge_index).relu()
#         x = self.conv2(x, edge_index).relu()
#         x = self.conv3(x, edge_index).relu()
#         x = torch.mean(x, dim=0)
#         x = self.fc(x)
#         return x

In [ ]:
import torch

# Danh sách kết nối giữa các node (khớp xương) dựa trên Mediapipe Pose
edges = [
    (0, 1), (1, 2), (2, 3), (0, 4), (4, 5), (5, 6), (0, 7), (0, 8), (7, 9), (8, 10),
    (11, 12), (11, 23), (12, 24), (23, 24), (11, 13), (13, 15), (15, 17), (15, 19), (15, 21),
    (12, 14), (14, 16), (16, 18), (16, 20), (16, 22), (23, 25), (25, 27), (27, 29), (29, 31),
    (24, 26), (26, 28), (28, 30), (30, 32)
]

# Chuyển thành tensor cho PyTorch Geometric
edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()

In [ ]:
def train_gcn(model, train_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs.squeeze(1), edge_index.to(device))
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    acc = 100 * correct / total
    return total_loss / len(train_loader), acc

In [ ]:
def evaluate_gcn(model, valid_loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in valid_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs.squeeze(1), edge_index.to(device))
            loss = criterion(outputs, labels)

            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    acc = 100 * correct / total
    return total_loss / len(valid_loader), acc

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = YogaGCN(in_channels=3, hidden_dim=64, num_classes=10).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
criterion = nn.CrossEntropyLoss()

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"📌 Số lượng tham số của YogaGCN: {count_parameters(model):,}")

In [ ]:
num_epochs = 100  # Số epoch

for epoch in range(num_epochs):
    train_loss, train_acc = train_gcn(model, train_loader, optimizer, criterion, device)
    valid_loss, valid_acc = evaluate_gcn(model, valid_loader, criterion, device)

    print(f"Epoch [{epoch+1}/{num_epochs}] | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}% | Valid Loss: {valid_loss:.4f} | Valid Acc: {valid_acc:.2f}%")

In [ ]:
torch.save(model.state_dict(), "yoga_gcn_model.pth")
print("✅ Mô hình đã được lưu thành công!")

In [ ]:
def predict_video(model, video_path, classes, device):
    model.eval()
    skeleton = np.load(video_path)  # Load skeleton đã trích xuất
    skeleton = torch.tensor(skeleton, dtype=torch.float32).to(device).unsqueeze(0)  # Thêm batch dim

    with torch.no_grad():
        output = model(skeleton.squeeze(1), edge_index.to(device))
        _, pred = torch.max(output, 1)
    
    return classes[pred.item()]

# Test với một video
video_path = "skeleton_data/test/Garland_Pose/video_13.npy"
predicted_class = predict_video(model, video_path, classes, device)
print(f"✅ Predicted Class: {predicted_class}")